In [1]:
from numpy import array
data = array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6])

In [2]:
import pandas as pd
df = pd.read_csv('titanic.csv')

In [3]:
df.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [5]:
X = df[['Pclass','SibSp','Parch','Fare']]
y = df.Survived

In [6]:
y.value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [7]:
X.head()

Pclass  SibSp  Parch     Fare
0       3      1      0   7.2500
1       1      1      0  71.2833
2       3      0      0   7.9250
3       1      1      0  53.1000
4       3      0      0   8.0500

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# SVM

In [9]:
from sklearn.svm import SVC
model3 = SVC()
model3.fit(X_train,y_train)
y_pred = model3.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)


0.6995515695067265

# 5-Fold Cross Validation

In [10]:
from sklearn.model_selection import cross_val_score,KFold
kf=KFold(n_splits=5)
score=cross_val_score(model3,X,y,cv=kf)
print("Cross Validation Scores are {}".format(score))

Cross Validation Scores are [0.62011173 0.66853933 0.66853933 0.66292135 0.70786517]


In [11]:
print("Average Cross Validation score :{}".format(score.mean()))

Average Cross Validation score :0.6655953800765803


# Random Search

In [12]:
parameters ={'C': [0.1, 1, 10, 100, 1000],
'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
'kernel': ['rbf']}

In [13]:
from sklearn.model_selection import RandomizedSearchCV
random_search=RandomizedSearchCV(estimator = SVC(), param_distributions=parameters,verbose=1, n_jobs=-1,
                            n_iter=20)
random_result = random_search.fit(X_train, y_train)
print('Best Score: ', random_result.best_score_*100)
print('Best Params: ', random_result.best_params_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Score:  70.21434182471104
Best Params:  {'kernel': 'rbf', 'gamma': 0.01, 'C': 1000}


# Hyperopt

In [14]:
#!pip install hyperopt

In [15]:
from hyperopt import tpe, hp, fmin, STATUS_OK,Trials

In [16]:
space = {'C': hp.choice('C', [0.1, 1, 10, 100, 1000]), 
              'gamma': hp.choice('gamma', [1, 0.1, 0.01, 0.001, 0.0001]),
              'kernel':  hp.choice('kernel', ['rbf'])} 

In [17]:
def hyperparameter_tuning(params):
    clf = SVC(**params)
    acc = cross_val_score(clf, X_train, y_train,scoring="accuracy").mean()
    return {"loss": -acc, "status": STATUS_OK}

In [18]:
trials = Trials()

best = fmin(
    fn=hyperparameter_tuning,
    space = space, 
    algo=tpe.suggest, 
    max_evals=50, 
    trials=trials
)

print("Best: {}".format(best))

100%|███████████████████████████████████████████████| 50/50 [01:37<00:00,  1.94s/trial, best loss: -0.7021434182471104]
Best: {'C': 4, 'gamma': 2, 'kernel': 0}
